In [1]:
# import the library
%matplotlib inline

import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
import seaborn as sns

from pyproj import Proj, transform

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import DBSCAN

import bokeh
import bokeh.plotting as plotting
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON
plotting.output_notebook()

sns.set_style('whitegrid')

Loading BokehJS ...

# Problem definition

Cluster regions based on crime data

http://donnees.ville.montreal.qc.ca/dataset/actes-criminels

# Load the data

In [2]:
#input
df = pd.read_csv('data/interventionscitoyendo.csv', encoding='latin_1')
df['DATE'] = pd.to_datetime(df['DATE'])
print(df.columns)
print(df['CATEGORIE'].value_counts())
df.head()

Index([u'CATEGORIE', u'DATE', u'QUART', u'PDQ', u'X', u'Y', u'LAT', u'LONG'], dtype='object')
Introduction                        30472
Vol dans / sur véhicule à moteur    24465
Méfait                              20702
Vol de véhicule à moteur            12453
Vols qualifiés                       4913
Infractions entrainant la mort         73
Name: CATEGORIE, dtype: int64


,CATEGORIE,DATE,QUART,PDQ,X,Y,LAT,LONG
0,Introduction,2015-01-01,jour,23,302375.197993,5.046522e+06,45.558606,-73.531060
1,Introduction,2015-01-01,jour,23,302375.197993,5.046522e+06,45.558606,-73.531060
2,Introduction,2015-01-01,soir,13,295850.656000,5.031730e+06,45.425443,-73.614364
3,Introduction,2015-01-01,nuit,8,289215.072000,5.036423e+06,45.467564,-73.699308
4,Introduction,2015-01-01,soir,44,298915.433995,5.046912e+06,45.562090,-73.575381


# Feature Engineering 

In [3]:
# feature engineering

# select a period
df = df[df['DATE']>='2018-01-01']

# select the categories
df = df[df['CATEGORIE']==u'Vols qualifiés']

# remove lines with no location
df = df[(df['X']>0)&(df['Y']>0)]

# adapt X and Y to the visualization
df['X'] = df.apply(lambda x: transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), x['LONG'], x['LAT'])[1], axis=1)
df['Y'] = df.apply(lambda x: transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), x['LONG'], x['LAT'])[0], axis=1)

X_columns = ['X', 'Y']
df = df[X_columns]

# Model Training

In [4]:
model = DBSCAN(eps=1000.0, min_samples=5)
model.fit(df[['X', 'Y']])

cluster_labels = model.labels_
n_clusters = len(set(cluster_labels))
print(collections.Counter(cluster_labels))

df['cluster'] = cluster_labels

Counter({-1: 161, 1: 84, 0: 7, 2: 7, 3: 6, 6: 6, 8: 6, 9: 6, 4: 5, 5: 5, 10: 5, 11: 5, 12: 5, 7: 4})


In [5]:
p = figure(y_range=(5641788.0, 5751788.0), x_range=(-8152883, -8252883))
p.add_tile(CARTODBPOSITRON)

latitude  = list(df[df['cluster']>-1]['X'].values)
longitude = list(df[df['cluster']>-1]['Y'].values)

colormap = list(bokeh.palettes.viridis(n_clusters))
colors = [colormap[x] for x in df[df['cluster']>-1]['cluster']]
source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude))
p.circle(x=longitude, y=latitude, color=colors, fill_alpha=0.2, size=5)
show(p)

# Model Evaluation

In [6]:
centroids = []
for cluster in sorted(set(cluster_labels)):
    centroids.append(df[df['cluster']==cluster][X_columns].mean().values)
print('Intra-Cluster Distances', sum(sum(euclidean_distances(centroids, centroids))))

distances = 0
for cluster in sorted(set(cluster_labels)):
    centroid = df[df['cluster']==cluster][X_columns].mean().values
    distances += (sum(euclidean_distances(df[df['cluster']==cluster][X_columns].values, centroid.reshape(1, -1))))[0]
print('Inter-Cluster Distances', distances)

('Intra-Cluster Distances', 2268302.4871340226)
('Inter-Cluster Distances', 1924666.6823221939)
